In [17]:
import os
import pandas as pd
import glob
import re

def processar_arquivos_vocalizacoes(pasta_origem):
    """
    Processa arquivos TXT com dados de vocalizações, mantendo apenas as 4 classificações
    especificadas e convertendo para códigos abreviados.
    """
    
    mapeamento_classificacao = {
        'phee adulto': 'p',
        'trill adulto': 't', 
        'phee bb': 'e',
        'trill bb': 'l'
    }
    
    # Usar pasta processados (não criar se já existir)
    pasta_processados = os.path.join(pasta_origem, "processados")
    
    padrao_arquivos = os.path.join(pasta_origem, '*.txt')
    arquivos_txt = glob.glob(padrao_arquivos)
    
    if not arquivos_txt:
        print(f"Nenhum arquivo TXT encontrado na pasta: {pasta_origem}")
        return
    
    print(f"Encontrados {len(arquivos_txt)} arquivo(s) TXT para processar.")
    print(f"Arquivos processados serão salvos em: {pasta_processados}")
    
    for caminho_arquivo in arquivos_txt:
        try:
            nome_arquivo = os.path.basename(caminho_arquivo)
            caminho_destino = os.path.join(pasta_processados, nome_arquivo)
            
            # Ler e processar o arquivo
            with open(caminho_arquivo, 'r', encoding='utf-8') as f:
                linhas = f.readlines()
            
            dados_processados = []
            linhas_ignoradas = 0
            
            for linha in linhas:
                linha = linha.strip()
                if not linha:  # Pular linhas vazias
                    continue
                
                # Dividir por qualquer espaço em branco
                partes = re.split(r'\s+', linha)
                
                if len(partes) >= 3:
                    # Juntar todas as partes além da 2ª para formar a classificação completa
                    inicio = partes[0]
                    fim = partes[1]
                    classificacao = ' '.join(partes[2:]).lower()
                    
                    # Verificar se é uma das classificações que queremos manter
                    if classificacao in mapeamento_classificacao:
                        codigo = mapeamento_classificacao[classificacao]
                        dados_processados.append([inicio, fim, codigo])
                    else:
                        linhas_ignoradas += 1
                else:
                    linhas_ignoradas += 1
            
            # Encontrar a largura máxima de cada coluna para melhor alinhamento
            max_inicio = max(len(str(item[0])) for item in dados_processados) if dados_processados else 8
            max_fim = max(len(str(item[1])) for item in dados_processados) if dados_processados else 6
            max_class = max(len(str(item[2])) for item in dados_processados) if dados_processados else 14
            
            # Ajustar para que o cabeçalho tenha pelo menos o tamanho dos nomes das colunas
            max_inicio = max(max_inicio, 6)   # "inicio" tem 6 caracteres
            max_fim = max(max_fim, 3)         # "fim" tem 3 caracteres
            max_class = max(max_class, 14)    # "classificacao" tem 14 caracteres
            
            # Salvar o arquivo processado COM cabeçalho perfeitamente alinhado
            with open(caminho_destino, 'w', encoding='utf-8') as f:
                # Escrever cabeçalho perfeitamente alinhado
                cabecalho = f"{'inicio':<{max_inicio}}\t{'fim':<{max_fim}}\t{'classificacao':<{max_class}}\n"
                f.write(cabecalho)
                
                # Escrever dados alinhados com o cabeçalho
                for inicio, fim, classificacao in dados_processados:
                    linha_formatada = f"{inicio:<{max_inicio}}\t{fim:<{max_fim}}\t{classificacao:<{max_class}}\n"
                    f.write(linha_formatada)
            
            # Estatísticas
            total_linhas_originais = len([l for l in linhas if l.strip()])
            total_linhas_processadas = len(dados_processados)
            
            print(f"Processado: {nome_arquivo}")
            print(f"  Linhas originais: {total_linhas_originais}")
            print(f"  Linhas mantidas: {total_linhas_processadas}")
            print(f"  Linhas ignoradas: {linhas_ignoradas}")
            
            if linhas_ignoradas > 0:
                print(f"  (Classificações não reconhecidas foram removidas)")
            
        except Exception as e:
            print(f"Erro ao processar {nome_arquivo}: {str(e)}")

def mostrar_classificacoes_arquivos(pasta_origem):
    """
    Mostra todas as classificações encontradas nos arquivos para referência
    """
    print("\n=== CLASSIFICAÇÕES ENCONTRADAS NOS ARQUIVOS ===")
    
    arquivos_txt = glob.glob(os.path.join(pasta_origem, '*.txt'))
    todas_classificacoes = set()
    
    for caminho_arquivo in arquivos_txt[:5]:  # Verifica apenas os primeiros 5 arquivos
        try:
            with open(caminho_arquivo, 'r', encoding='utf-8') as f:
                for linha in f:
                    partes = linha.strip().split()
                    if len(partes) >= 3:
                        classificacao = ' '.join(partes[2:]).lower()
                        todas_classificacoes.add(classificacao)
        except:
            pass
    
    print("Classificações encontradas:")
    for cls in sorted(todas_classificacoes):
        status = "✓ Mapeada" if cls in ['phee adulto', 'trill adulto', 'phee bb', 'trill bb'] else "✗ Ignorada"
        print(f"  {cls} -> {status}")

if __name__ == "__main__":
    caminho_pasta = input("Digite o caminho completo da pasta com os arquivos TXT: ")
    
    if not os.path.exists(caminho_pasta):
        print("A pasta especificada não existe!")
    else:
        # Primeiro mostra quais classificações existem
        mostrar_classificacoes_arquivos(caminho_pasta)
        
        # Verificar se pasta processados existe
        pasta_processados = os.path.join(caminho_pasta, "processados")
        if not os.path.exists(pasta_processados):
            print(f"\nAviso: A pasta 'processados' não existe em {caminho_pasta}")
            print("O processamento será cancelado.")
        else:
            # Pergunta se quer continuar
            print("\nDeseja continuar com o processamento?")
            resposta = input("Pressione ENTER para SIM ou ESPAÇO + ENTER para NÃO: ")
            
            if resposta.strip() == "":  # Enter sem digitar nada = SIM
                processar_arquivos_vocalizacoes(caminho_pasta)
                print("\nProcessamento concluído!")
                print("Os arquivos processados foram salvos na pasta 'processados'")
                print("Os arquivos incluem cabeçalho perfeitamente alinhado com as colunas:")
                print("  - inicio")
                print("  - fim") 
                print("  - classificacao")
            else:
                print("Processamento cancelado.")


=== CLASSIFICAÇÕES ENCONTRADAS NOS ARQUIVOS ===
Classificações encontradas:
  choro -> ✗ Ignorada
  phee bb -> ✓ Mapeada
  phii -> ✗ Ignorada
  trill adulto -> ✓ Mapeada
  trill bb -> ✓ Mapeada

Deseja continuar com o processamento?
Encontrados 7 arquivo(s) TXT para processar.
Arquivos processados serão salvos em: C:\Users\Ricardo\Desktop\Gravador 01-20250917T162913Z-1-001\Gravador 01\processados
Processado: 20220826_145000.txt
  Linhas originais: 1
  Linhas mantidas: 1
  Linhas ignoradas: 0
Processado: 20220826_145200.txt
  Linhas originais: 1
  Linhas mantidas: 1
  Linhas ignoradas: 0
Processado: 20220826_145400.txt
  Linhas originais: 1
  Linhas mantidas: 1
  Linhas ignoradas: 0
Processado: 20220826_145800.txt
  Linhas originais: 1
  Linhas mantidas: 1
  Linhas ignoradas: 0
Processado: 20220826_150200.txt
  Linhas originais: 21
  Linhas mantidas: 13
  Linhas ignoradas: 8
  (Classificações não reconhecidas foram removidas)
Processado: 20220829_153400.txt
  Linhas originais: 1
  Linh